In [ ]:
import pandas as pd
import os
import sys

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
df = pd.read_csv(csv_path)

# Observe the types of all the columns 
print("=" * 50)
print(df.dtypes)

# Convert to pandas datetime and normalize to date (removes time component)
# Keeping as pandas datetime (not Python date) for pandas operations like .dt.year
df['date'] = pd.to_datetime(df['window_start'], unit='ns').dt.normalize()

if 'ticker' in df.columns:
        df['ticker'] = df['ticker'].astype('category')

# Rename window_start to unix_nsec_timestamp
df = df.rename(columns={'window_start': 'unix_nsec_timestamp'})

# Reorder columns to match desired order: date, unix_nsec_timestamp, ticker, open, close, high, low, volume, transactions
desired_order = ['date', 'unix_nsec_timestamp', 'ticker', 'open', 'close', 'high', 'low', 'volume', 'transactions']
df = df[desired_order]


print("=" * 50)
print(df.head())
print(df.dtypes)


# Check for NaN values in ticker column
print(f"NaN count: {df['ticker'].isna().sum()}")
print(f"NaN percentage: {df['ticker'].isna().sum() / len(df) * 100:.2f}%")
print(f"Unique tickers (with NaN): {len(df['ticker'].unique())}")
print(f"Unique tickers (without NaN): {len(df['ticker'].dropna().unique())}")

In [ ]:
import pandas as pd
import os
import sys
from typing import List, Tuple, Dict, Any
from datetime import date


# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
df = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date (removes time component)
df['date'] = pd.to_datetime(df['window_start'], unit='ns').dt.normalize()

def build_payload(
	df: pd.DataFrame,
	threshold: int
	) -> List[Tuple[str, date]]:
	""" 
	Builds the data that we want to send to the API

	We will be sending a ticker and date, 
	"""
	
	# Get each unique ticker with its first and last date
	ticker_dates = df.groupby('ticker').agg(
		first_date=('date', 'min'),
		last_date=('date', 'max')
	).reset_index()

	# Generate list of (ticker, date) tuples for each ticker from first_date to first_date + threshold
	list_of_calls = []

	# Now fill in dates for each ticker group with the threshold we choose
	for _, row in ticker_dates.iterrows():
		ticker = row['ticker']
		first_date = pd.to_datetime(row['first_date'])
		last_date = pd.to_datetime(row['last_date'])
		
		# Generate dates from first_date to end_date, with threshold as the interval
		date_range = pd.date_range(
			start=first_date,
			end=last_date,
			freq=f'{threshold}D'
		)
		
		# Add (ticker, date) tuple for each date
		for date in date_range:
			list_of_calls.append((ticker, date.date()))
	
	return list_of_calls

list = build_payload(df, 14)

for item in list:
    print(item[0], item[1])

In [ ]:
# Check the distribution of tickers that have very little continuous trading data and determine a threshold to filter out, perhaps less than 30 days

import pandas as pd
import os
import sys

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
df = pd.read_csv(csv_path)

df['date'] = pd.to_datetime(df['window_start'], unit='ns').dt.normalize()


tickers_too_little_data = []

for ticker, group in df.groupby('ticker', observed=True):
    dates = group['date'].unique()

    if len(dates) < 30:
        tickers_too_little_data.append(ticker)

print(len(tickers_too_little_data))
print(tickers_too_little_data)

In [ ]:
# Performance Test: String (object) vs Category dtype for ticker column
import pandas as pd
import numpy as np
import time

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
df_test = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date
df_test['date'] = pd.to_datetime(df_test['window_start'], unit='ns').dt.normalize()

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in df_test.columns if col != 'date']
df_test = df_test[cols]

print("=" * 70)
print("DATASET INFO")
print("=" * 70)
print(f"Total rows: {len(df_test):,}")
print(f"Unique tickers: {df_test['ticker'].nunique():,}")
print(f"Average rows per ticker: {len(df_test) / df_test['ticker'].nunique():.1f}")
print(f"\nMemory usage (object dtype):")
mem_obj = df_test['ticker'].memory_usage(deep=True) / 1024**2
print(f"{mem_obj:.2f} MB")

# Create a copy with category dtype
df_cat = df_test.copy()
df_cat['ticker'] = df_cat['ticker'].astype('category')

print(f"\nMemory usage (category dtype):")
mem_cat = df_cat['ticker'].memory_usage(deep=True) / 1024**2
print(f"{mem_cat:.2f} MB")
print(f"Memory savings: {(1 - mem_cat / mem_obj) * 100:.1f}%")

print("\n" + "=" * 70)
print("PERFORMANCE TESTS")
print("=" * 70)

# Test 1: groupby operations
print("\n1. GROUPBY OPERATIONS")
print("-" * 70)

# Object dtype
start = time.time()
result_obj = df_test.groupby('ticker')['volume'].sum()
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
result_cat = df_cat.groupby('ticker')['volume'].sum()
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

# Test 2: Filtering with isin()
print("\n2. FILTERING WITH isin()")
print("-" * 70)

# Get a sample of tickers to filter
sample_tickers = df_test['ticker'].unique()[:1000].tolist()

# Object dtype
start = time.time()
filtered_obj = df_test[df_test['ticker'].isin(sample_tickers)]
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
filtered_cat = df_cat[df_cat['ticker'].isin(sample_tickers)]
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

# Test 3: Sorting
print("\n3. SORTING")
print("-" * 70)

# Object dtype
start = time.time()
sorted_obj = df_test.sort_values('ticker')
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
sorted_cat = df_cat.sort_values('ticker')
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

# Test 4: value_counts()
print("\n4. value_counts()")
print("-" * 70)

# Object dtype
start = time.time()
vc_obj = df_test['ticker'].value_counts()
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
vc_cat = df_cat['ticker'].value_counts()
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

# Test 5: set_index with ticker
print("\n5. set_index() with ticker")
print("-" * 70)

# Object dtype
start = time.time()
idx_obj = df_test.set_index(['ticker', 'date'])
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
idx_cat = df_cat.set_index(['ticker', 'date'])
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

# Test 6: Aggregation operations
print("\n6. MULTIPLE AGGREGATIONS")
print("-" * 70)

# Object dtype
start = time.time()
agg_obj = df_test.groupby('ticker').agg({
    'volume': ['sum', 'mean', 'max'],
    'close': ['mean', 'std']
})
time_obj = time.time() - start
print(f"Object dtype: {time_obj:.4f} seconds")

# Category dtype
start = time.time()
agg_cat = df_cat.groupby('ticker').agg({
    'volume': ['sum', 'mean', 'max'],
    'close': ['mean', 'std']
})
time_cat = time.time() - start
print(f"Category dtype: {time_cat:.4f} seconds")
print(f"Speedup: {time_obj / time_cat:.2f}x")

In [ ]:
# Analyzing how much of the data (or haw many ticker series) have at MAX 100 shares vol traded on a day through their ENTIRE series 
# and never reached above $0.01 on their ENTIRE series. 

# Goal is to potentially remove some untradeable noise, but first to see how much of that noise we would be removing

import re
from collections import defaultdict
import pandas as pd
import numpy as np

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
OHLCV_data = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date (removes time component)
# Keeping as pandas datetime (not Python date) for pandas operations like .dt.year
OHLCV_data['date'] = pd.to_datetime(OHLCV_data['window_start'], unit='ns').dt.normalize()

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in OHLCV_data.columns if col != 'date']
OHLCV_data[cols]

unique_tickers = OHLCV_data['ticker'].unique()

# Find and analyze the number of tickers that have MAX 100 shares volume across their entire series 
# Compute max volume and max price per ticker in one pass
ticker_stats = OHLCV_data.groupby('ticker').agg({
    'volume': 'max',
    'close': 'max'
}).reset_index()

# See what the distribution of price and volume is across our series
print(ticker_stats['volume'].min())
print(ticker_stats['volume'].quantile([0.01, 0.05, 0.10, 0.25, 0.50]))
print(ticker_stats['close'].min())

# Filter tickers into a list that are below the criteria
invalid_tickers = ticker_stats[
    (ticker_stats['volume'] < 1000) |
    (ticker_stats['close'] < 0.01)
]['ticker'].tolist()

print(invalid_tickers)
print(len(invalid_tickers))

In [ ]:
# Look at all of the suffixes we have for the unique tickers
import re
from collections import defaultdict
import pandas as pd
import numpy as np

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
OHLCV_data = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date (removes time component)
# Keeping as pandas datetime (not Python date) for pandas operations like .dt.year
OHLCV_data['date'] = pd.to_datetime(OHLCV_data['window_start'], unit='ns').dt.normalize()

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in OHLCV_data.columns if col != 'date']
OHLCV_data[cols]

# Get unique tickers and filter out NaN values, convert to strings
unique_tickers = OHLCV_data['ticker'].dropna().unique()
unique_tickers = [str(ticker) for ticker in unique_tickers]

# Function to extract suffix from a ticker
def extract_suffix(ticker):
    """
    Extract suffix from ticker. Returns (base, suffix) or (ticker, None) if no suffix detected.
    Suffixes are typically 1-3 characters at the end that are not digits.
    """

    # Match test tickers (case-insensitive)
    if re.search(r'(?i)test', ticker):
        # Handle test tickers
        return ticker, 'TEST'

    # Match ZVZZT/ZWZZT test tickers
    if re.match(r'^(ZVZZT|ZWZZT)$', ticker):
        return ticker, None

    # Match non-equities (lowercase/period suffixes)
    non_equities_match = re.match(r'^([^a-z.]*)([a-z.].*)$', ticker)
    if non_equities_match:
        base, suffix = non_equities_match.groups()
        if len(base) >= 1 and len(suffix) >= 1:
            return base, suffix

    # If no pattern matches, return the whole ticker as base with no suffix
    return ticker, None

# Group tickers by suffix - simple dictionary mapping suffix -> list of tickers
suffix_groups = defaultdict(list)

for ticker in unique_tickers:
    base, suffix = extract_suffix(ticker)
    if suffix:
        suffix_groups[suffix].append(ticker)
    else:
        # Tickers with no detected suffix go under a no_suffix column
        suffix_groups['no suffix'].append(ticker)


# Find the maximum length needed for DataFrame
max_len = max(len(tickers) for tickers in suffix_groups.values()) if suffix_groups else 0

# Pad all lists to the same length with NaN
for suffix in suffix_groups:
    suffix_groups[suffix] = suffix_groups[suffix] + [np.nan] * (max_len - len(suffix_groups[suffix]))

# Create DataFrame
suffix_df = pd.DataFrame(dict(sorted(suffix_groups.items())))

# take a look at the dataframe
print(suffix_df)

# Dictionary to store the column names (suffixes) and the COUNT of tickers that fall under that suffix 
# we dont currently use the dictionary but could be nice to have
suffix_dict = {}

for column_name, column_data in suffix_df.items():
    suffix_dict[column_name] = column_data.count()
    print(f"{column_name} : {column_data.count()}")

# number of non suffixed unique tickers
count_no_suffix = suffix_dict['no suffix']
# number of uniquely suffix tickers identified by the regex
count_suffix = len(unique_tickers) - count_no_suffix

# Percent of uniquely suffixed tickers for all unique tickers 
# Will be removinig these 
print(count_suffix / len(unique_tickers))

In [ ]:
# We can also remove rights, warrants and units confidently by checking for 5 lengthed tickers with U, W, or R as their 5th char
# that have a 4 char lengthed ticker, 
# we dont want to try and simply remove tickers based on them having a U, R, W at the end of their string, but if 
# there is another ticker that is an exact match up to the final character, and the final character is a W, U or R
# we can confidently remove those tickers, keeping on the vanilla equity.

# Look at all of the suffixes we have for the unique tickers
import re
from collections import defaultdict
import pandas as pd
import numpy as np

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
df = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date (removes time component)
# Keeping as pandas datetime (not Python date) for pandas operations like .dt.year
df['date'] = pd.to_datetime(df['window_start'], unit='ns').dt.normalize()

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in df.columns if col != 'date']
df[cols]


def sanitize_warrants_rights_units(
    df: pd.DataFrame
    ) -> pd.DataFrame:
    """
    Removes warrants, rights, and units by checking for 5-character tickers ending in U, W, or R
    that have a matching 4-character base ticker.
    """
    unique_tickers = set(df['ticker'].dropna().unique())
    tickers_to_remove = []

    for ticker in unique_tickers:
        if isinstance(ticker, str) and len(ticker) == 5 and ticker[4] in ['U', 'W', 'R']:
            base_ticker = ticker[:4]
            if base_ticker in unique_tickers:
                tickers_to_remove.append(ticker)

    mask_to_keep = ~df['ticker'].isin(tickers_to_remove)
    return df[mask_to_keep], tickers_to_remove

df, tickers_to_remove  = sanitize_warrants_rights_units(OHLCV_data)

print(len(tickers_to_remove))
for ticker in tickers_to_remove:
    print(ticker)
    

In [ ]:
# Analyzing duplicates
import re
from collections import defaultdict
import pandas as pd
import numpy as np

# Load in OHLCV Data 
csv_path = r'C:\Users\carso\Development\emerytrading\Data\Stocks\Polygon\OHLCV_Historical_2016-01-01_to_2025-10-26.csv'
OHLCV_data = pd.read_csv(csv_path)

# Convert to pandas datetime and normalize to date (removes time component)
# Keeping as pandas datetime (not Python date) for pandas operations like .dt.year
OHLCV_data['date'] = pd.to_datetime(OHLCV_data['window_start'], unit='ns').dt.normalize()

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in OHLCV_data.columns if col != 'date']
OHLCV_data[cols]

# Find the duplcate rows
duplicate_rows = OHLCV_data[OHLCV_data.duplicated(subset=['ticker', 'date'], keep=False)]
print(type(duplicate_rows))

# Select only 'ticker' and 'date' columns, sort by ticker
dupes = duplicate_rows[['ticker', 'date', 'close','open', 'volume']].sort_values(by='ticker')

print(dupes)
print(len(dupes))
# print(len(duplicate_rows))